In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lbg_desc_forecast import (
    get_lbg_mappers,
    Forecaster,
    MainCosmology,
    data_dir,
    fig_dir,
)

In [ ]:
# Create forecaster that will create the signal
forecaster = Forecaster(
    get_lbg_mappers(10),
    MainCosmology(),
)

In [ ]:
# Create signal
signal = forecaster.create_signal(add_noise=False)

# Load pre-saved covariance
cov = np.load(data_dir / "signal_covariance.npy")

In [ ]:
# Get ells for bins
ell = forecaster.bins.get_effective_ells()

# Unpack signal
Cgg = dict(
    u=signal[:20],
    g=signal[40:60],
    r=signal[80:100],
)
Ckg = dict(
    u=signal[20:40],
    g=signal[60:80],
    r=signal[100:120],
)

# Unpack variances
err = np.sqrt(np.diag(cov))
err_gg = dict(
    u=err[:20],
    g=err[40:60],
    r=err[80:100],
)
err_kg = dict(
    u=err[20:40],
    g=err[60:80],
    r=err[100:120],
)

In [ ]:
fig, axes = plt.subplots(
    1, 3, figsize=(6, 2.3), constrained_layout=True, dpi=150, sharex=True, sharey=True
)

for ax, band, z, color in zip(axes, "ugr", [3, 4, 5], ["C4", "C0", "C2"]):
    cs = 2.2
    ax.errorbar(
        ell,
        Cgg[band],
        yerr=err_gg[band],
        label="$C^{gg}$ ",
        c=color,
        ls="",
        marker="",
        capsize=cs,
        lw=0.75,
        markersize=3,
    )
    ax.errorbar(
        ell,
        Ckg[band],
        yerr=err_kg[band],
        label="$C^{\kappa g}$ ",
        c=color,
        ls="",
        marker="_",
        markersize=3,
        capsize=cs,
        lw=0.75,
    )
    ax.legend(
        fontsize=8,
        handlelength=1,
        borderpad=0,
        frameon=False,
        loc="lower left",
        handletextpad=0.2,
    )

    snr_gg = np.sqrt(np.sum(Cgg[band] ** 2 / err_gg[band] ** 2))
    snr_kg = np.sqrt(np.sum(Ckg[band] ** 2 / err_kg[band] ** 2))
    ax.text(
        0.95,
        0.95,
        "SNR$^{gg} = $ " + f"${snr_gg:.0f}$",
        ha="right",
        va="top",
        transform=ax.transAxes,
        fontsize=8,
    )
    ax.text(
        0.925,
        0.85,
        "SNR$^{\kappa g} = $ " + f"${snr_kg:.0f}$",
        ha="right",
        va="top",
        transform=ax.transAxes,
        fontsize=8,
    )
    ax.set(title=f"$z \sim {z}$", xlabel="$\ell$")

axes[0].set(yscale="log", xscale="log", ylabel="$C_\ell$")

fig.savefig(fig_dir / "power_spectra.pdf", bbox_inches="tight")